# Network Formation with Hamiltonian Markov Chain Monte Carlo





## Introduction

This project analyzes the formation and structure of trust networks in historical financial markets, using data from the New York Stock Exchange (NYSE) spanning 1883 to 1930. We explore how social connections, ethnicity, and latent characteristics influenced the sponsorship of new NYSE members and the subsequent approval process by existing members.

## Stein prep

In [ ]:
try:
    from google.colab import drive
    import os
    drive.mount('/content/drive')
    os.chdir('drive/MyDrive/School/DS-GA 1006/code')
    print(os.getcwd())
except:
  pass

In [ ]:
# ! pip install -r requirements.txt

### Importing Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import tensorflow_probability as tfp
from tqdm import tqdm
from functools import lru_cache
from collections import defaultdict

### Constants

In [ ]:
tf.executing_eagerly()
import time
from functools import wraps

def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} took {total_time:.4f} seconds')
        return result
    return wrapper

## Data

Based on the provided data structure and the model described in the document, we can formulate an expressive representation of the limiting likelihood function. Let's define the key components:

1. Node attributes:
$x_i$ = ($ethnicity_i$, $everCommittee_i$, $everSponsor_i$)
2. Edge formation: Let $L_{ij,t}$ be an indicator for whether node i chose node j as a sponsor in transaction t.
3. Payoff structure:
$U_{ij,t}$ = $U^*(x_i, x_j; s_{i,t}, s_{jt}) + σε_{ij,t}$
    Where:
    - $s_{it}$, $s_{jt}$ are node-specific network statistics (e.g., degree centrality)

4. Aggregate state variables:
    - $H^*(x; s)$: Inclusive value function, which is a fixed point condition we subject the log-lieklihood to inorder to avoid double counting of non zero links in the network.
    - $H^*(x_i, s_{it}) = \frac{1}{n}∑_{k=1}^{n}Ψ_k(\theta, H^*(x; s))$
    - $w_i = \frac{\mathbb{1}[s_{i,1} > 0]}{\frac{1}{n}∑_{k=1}^{n}\mathbb{1}(s_{k,1}>0) } $
    - **Psi function**: $ Ψ_i (\theta,H) = w_i s_{i,1}* \frac{exp[V(x, x_i, \theta)]}{1+H(x_i)}$
    - $M^*(x_i, x_j, s_{it})$: The degree distribution of the network
    - $M^*(x_i, x_j, s_{it}) = \frac{H*(x, s)}{1+H^*(x, s+1)}$
    
5. Limiting link frequency probability:
$$μ_0(L_{ij,t} = 1, s_{it}, s_{jt} | x_i, x_j) = \frac{s_{it} s_{ij,t} *exp(U^*(x_i,x_j;s_{it}, s_{jt}) + U^*(x_j,x_i;s_{jt},s_{it}))}{(1 + H^*(x_i,s_{it}))*(1 + H^*(x_j,s_{jt}))}$$

$$log{μ_0(L_{ij,t} = 1, s_{it}, s_{jt} | x_i, x_j)} = log{{s_{it}} + log{s_{ij,t}} + U^*(x_i,x_j;s_{it}, s_{jt}) + U^*(x_j,x_i;s_{jt},s_{it}) - log{1 + H^*(x_i,s_{it})} - log{1 + H^*(x_j,s_{jt})}$$

6. Committee voting: Let $v_{kt}$ be the vote (white ball or black ball) of committee member k in transaction t. $$P(v_{kt} = white | x_i, x_{j1}, x_{j2}, x_k) = Φ(αx_i + β_1x_{j1} + β_2x_{j2} + γx_k + δ_1d(Z_i, Z_k) + δ_2d(Z_{j1}, Z_k) + δ_3d(Z_{j2}, Z_k))$$
Where:
    - Φ is the standard normal CDF
    - $Z_i$, $Z_{j1}$, $Z_{j2}$, $Z_k$ are latent positions in a low-dimensional Euclidean space
    d(·,·) is a distance function in this space.


In [ ]:
# Read the data files
node_data = pd.read_csv('data/nyse_node_sp1.csv', header=None,
                        names=['name', 'ever_committee', 'node_id', 'ethnicity', 'ever_sponsor'])
edge_data = pd.read_csv('data/nyse_edge_buy_sp_sp1.csv', header=None,
                        names=['buyer_id', 'sponsor1_id', 'sponsor2_id', 'f1', 'f2', 'f3', 'f4', 'blackballs', 'whiteballs', 'year'])
committee_data = pd.read_csv('data/nyse_edge_buy_com1.csv', header=None,
                             names=['buyer_id', 'committee_id', 'f1', 'f2', 'f3', 'f4', 'blackballs', 'whiteballs', 'year'])



FileNotFoundError: [Errno 2] No such file or directory: 'data/nyse_node_sp1.csv'

In [ ]:
def process_data(node_data, edge_data, committee_data):
    node_data = pd.get_dummies(data=node_data, columns=['ethnicity'], dummy_na=True, prefix='ethnicity', drop_first=True, dtype=int)
    node_data[['ever_committee', 'ever_sponsor']] = node_data[['ever_committee', 'ever_sponsor']].fillna(0)
    node_attrs = node_data.set_index('node_id').drop(columns=['name']).T.to_dict('list')

    # Initialize network statistics
    network_stats = {node_id: {'degree': 0, 'sponsor_count': 0} for node_id in node_attrs}
    edges = defaultdict(set)

    transactions = []
    for _, row in edge_data.iterrows():
        buyer_id = row['buyer_id']
        sponsor1_id = row['sponsor1_id']
        sponsor2_id = row['sponsor2_id']
        year = row['year']

        # Update network statistics
        network_stats[buyer_id]['degree'] += 2
        network_stats[sponsor1_id]['degree'] += 1
        network_stats[sponsor2_id]['degree'] += 1
        network_stats[sponsor1_id]['sponsor_count'] += 1
        network_stats[sponsor2_id]['sponsor_count'] += 1
        edges[buyer_id].add(sponsor1_id)
        edges[buyer_id].add(sponsor2_id)

        committee_members = committee_data[(committee_data['buyer_id'] == buyer_id) &
                                           (committee_data['year'] == year)]['committee_id'].tolist()

        transactions.append({
            'buyer_id': buyer_id,
            'sponsor1_id': sponsor1_id,
            'sponsor2_id': sponsor2_id,
            'committee_members': committee_members,
            'year': year,
            'whiteballs': row['whiteballs'],
            'blackballs': row['blackballs']
        })

    return node_attrs, transactions, network_stats, edges

node_attrs, transactions, network_stats, edges = process_data(node_data, edge_data, committee_data)
node_stats = {k: v['degree'] for k, v in network_stats.items()}

## Model

**Importance weights**
$$ w_i = \frac{\mathbb{1}[s_{i,1} > 0]}{\frac{1}{n}∑_{k=1}^{n}\mathbb{1}(s_{k,1}>0) } $$
**Psi function**
$$ Ψ_i (\theta,H) = w_i s_{i,1}* \frac{exp[V(x, x_i, \theta)]}{1+H(x_i)}$$

**Inclusive Value Function**: This is a fixed point condition we subject the log-lieklihood to inorder to avoid double counting of non zero links in the network.
$$ H(x) = \frac{1}{n}∑_{i=1}^{n}\psi_i(\theta,H)$$

In [ ]:

class CachedVStar:
    def __init__(self, maxsize=10000):
        self.cache = {}
        self.maxsize = maxsize

    def _create_symmetric_key(self, xi, xj, si, sj, theta):
        # Create two possible arrangements
        t = list(map(str, theta))
        key1 = '_'.join(list(map(str, xi)) + list(map(str, xj)) + [str(si), str(sj)] + t)
        key2 = '_'.join(list(map(str, xj)) + list(map(str, xi)) + [str(sj), str(si)] + t)

        # Return the smaller key to ensure consistency
        return min(key1, key2)

    @tf.function#(reduce_retracing=True)
    def U_star(self, xi, xj, si, sj, theta):
        # Define utility function
        input = tf.convert_to_tensor(list(xi) + list(xj) + [si] + [sj], dtype=tf.float32)
        theta = tf.convert_to_tensor(theta, dtype=tf.float32)
        return tf.tensordot(theta, input, axes=1)

    # Define pseudo-surplus function (section 4.2 pg 34)
    @tf.function#(reduce_retracing=True)
    def V_star(self, xi, xj, si, sj, theta):
        return self.U_star(xi, xj, si, sj, theta) + self.U_star(xj, xi, sj, si, theta)

    def __call__(self, xi, xj, si, sj, theta):
        key = self._create_symmetric_key(xi, xj, si, sj, theta)
        if key not in self.cache:
            if len(self.cache) >= self.maxsize:
                # Remove oldest entry
                self.cache.pop(next(iter(self.cache)))
            self.cache[key] = self.V_star(xi, xj, si, sj, theta)
        return self.cache[key]

# Defined the importance weight function (section 4.2.1 pg 36)
@lru_cache(maxsize=10)
def calculate_weights(s, node_vals):
    t = tf.convert_to_tensor(list(node_vals))
    indicator_tensor = tf.cast(tf.greater(t, 0), tf.float32)
    denominator = tf.reduce_mean(indicator_tensor).numpy()
    return s / denominator

# Define Psi function (section 4.2.1 pg 36)
@tf.function
def psi_i( H, x_i, s_i1, w_i, V):
    num = tf.exp(V)
    denom = 1 + H(x_i)
    res = w_i * s_i1 * num / denom
    return res

# define the inclusive value function
@tf.function
def H_star(node_attrs, node_stats, weights, theta, H, v_star, max_iter=10):

    # Initialize H(x) with zeros
    H_current = H

    for i in tqdm(range(max_iter), desc='H_star', position=0, leave=True):
        H_prev = H_current

        psi_mean = defaultdict(list)
        unique_node_pairs = set((tuple(node_attrs[node_id]), node_stats[node_id])
                           for node_id in node_attrs.keys())
        for x, s in tqdm(unique_node_pairs, desc='processing unique (x,s) pairs', position=1, leave=True):
            psi_values = []
            for i in node_attrs.keys():
                x_i, s_i = node_attrs[i], node_stats[i]
                V = v_star(x, x_i, s, s_i, theta)
                psi_i_value = psi_i( H_prev, x_i, s_i, weights[i], V)
                psi_values.append(psi_i_value)

            psi_mean[str(x)].extend(psi_values)
        H_current = lambda x: tf.reduce_mean(tf.stack(psi_mean[str(x)]), axis=0)

        # Check convergence - now comparing full tensors
        diff = tf.reduce_max(tf.abs(H_current(x) - H_prev(x)))
        if diff < 1e-3:
            print(f"Convergence achieved after {i} iterations.")
            break

    return H_current

# Log likelihood contribution for node_id
def log_likelihood_contribution(Lijt, x_i, x_j, s_i, s_j, theta, H, v_star):
    V = v_star(x_i, x_j, s_i, s_j, theta)
    ll = 0.5 * Lijt * (V - tf.math.log(1+ H(x_i)) - tf.math.log(1 + H(x_j)))
    return ll


In [ ]:
# Log limiting likelihood function
def log_likelihood(theta, node_attrs, node_stats, edges, weights, H = None):

    # Initialize H if not given and loglikelihood
    print('Cache Vstar')
    cached_v_star = CachedVStar()
    H = H if H is not None else lambda x: tf.zeros_like(x)
    H = H_star(node_attrs, node_stats, weights, theta, H, cached_v_star)
    print('Calculated H')
    l_list = []
    ll = 0.0
    print('Calculating log likelihood')
    for i in tqdm(node_attrs.keys(), desc='Log likelihood', position=0, leave=True):
        x_i, s_i = node_attrs[i], node_stats[i]
        for j in edges[i]:
            if i == j:
                continue
            x_j, s_j = node_attrs[j], node_stats[j]
            ll += log_likelihood_contribution(tf.constant(1.0), x_i, x_j, s_i, s_j, theta, H, cached_v_star)
        ll += tf.math.log(float(s_i)) - tf.math.log(1 + H(x_i))
        l_list.append(ll)
    return ll, H, l_list



## Inference

In [ ]:
# Set up HMC
num_results = 1000
num_burnin_steps = 1000

# Initialize parameters
x = len(list(node_attrs.values())[0])
initial_state = [0]*x*2 + [0, 0]  #x_i, x_j , si, sj

# initial_state = tf.Variable(initial_state)

# Define the HMC transition kernel
step_size = tf.Variable(0.01)
adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
    tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=log_likelihood,
        num_leapfrog_steps=10,
        step_size=step_size),
    num_adaptation_steps=int(num_burnin_steps * 0.8))

In [ ]:
weights = {node_id: calculate_weights(s > 0, node_stats.values()) for node_id, s in node_stats.items() }

ll, H, l_list = log_likelihood(initial_state, node_attrs, node_stats, edges, weights)
ll

In [ ]:
import seaborn as sns
sns.lineplot(data=l_list)